In [102]:
import numpy as np
import pandas as pd
import math

In [192]:
def find_n(k):
    # Initial guess for n
    n = k + 2  # A simple starting guess that should be larger than k + 1
    while True:
        next_n = k + 1 + math.ceil(math.log2(n))
        if next_n == n:
            return n
        n = next_n
    

# Calculate the sum of bit positions as specified
def sum_positions(v):
    return sum((i+1) * bit for i, bit in enumerate(v) if bit is not None)

In [193]:
def encoder_VT(u, a):
  k = len(u)
  n = find_n(k)
  sum = 0
  # Initialize the codeword with None to indicate positions to be filled
  v = [None] * n

  # find out the parity indexes which we insert
  parity_index = [2**i for i in range(0,n-k-1)]

  if (parity_index[-1] < n):
    parity_index.append(n)
  # set non parity outputs and sum the existing bits in their positions
  sum = 0
  j = 0
  for i in range(1,n+1):
    if i not in parity_index:
      v[i-1] = u[j]
      sum += i*int(u[j])
      j += 1
  #find out what the sum of the unknown parity bits should be (including modulo)
  m = 2*n+1
  i = 0
  while (m*i+a-sum < 0): # finding the lowest sum which satisfies the equation
    i+=1
  parity_sum = m*i+a-sum

  # find out parity bits by subtracting position
  for i in range(len(parity_index)-1,-1,-1):
    if (parity_sum >= parity_index[i]):
      parity_sum -= parity_index[i]
      v[parity_index[i]-1] = '1'
    else:
      v[parity_index[i]-1] = '0'
  return ''.join(v)

In [194]:
def extract_message(v):
  n = len(v)
  k = n- math.ceil(math.log2(n))-1
  u = [None]*k
  # find out the parity indexes which we insert
  j = 0
  for i in range(0,n-1):
    if (math.log2(i+1) != math.floor(math.log2(i+1))) & (j < k):
      u[j] = v[i]
      j += 1
  return ''.join(u)


In [195]:
extract_message(encoder_VT("1111",a=0))

'1111'

In [196]:
p_d, p_i, p_s  = 1/4,1/4,1/4 #probabilities
p_t = 1-p_d-p_i #probability of transition

In [197]:
def gamma(r,t,d,d_tag,b): #gamma assuming we setup s correctly
  l = d-d_tag
  if l== -1:
    return p_d
  if l>=0:
    if d+t-1 >= len(r) or d+t-1 < 0:
      #  print("hi")
       return 0 # out of bounds
    # Note: the sequence starts from t=1 so we need to subtract 1
    if int(r[d+t-1]) != b:
      return (0.5*p_i)**l*(p_t*p_s+0.5*p_i*p_d)
    else:
      return (0.5*p_i)**l*(p_t*(1-p_s)+0.5*p_i*p_d)
  else: return 0


In [198]:
gamma("10101",4,0,0,0)

0.40625

In [251]:
# # given probabilities, find the additional range of looking at values.
# A = (p_t*p_s+0.5*p_i*p_d)
# B = p_t*(1-p_s)+0.5*p_i*p_d
# C = 0.5*p_i

# if (A/B == 0 ):
#   additional_range = 0
# else:
#   additional_range = np.floor(np.abs(np.emath.logn(C,A/B))).astype(int)
# print(additional_range)

In [252]:
def print_strand(v, pi, pd, ps):
    # Check that probabilities sum to 1
    r = []

    for vi in v:
        while True:
            rand = np.random.random()
            if rand < pi:
                # Insertion error
                r.append(np.random.randint(2))  # Append a random 0 or 1
            else:
                # No more insertions, break the loop
                break

        rand = np.random.random()
        if rand < pd:
            # Deletion error
            continue  # vi is deleted, so skip appending vi
        else:
            # Transmission (correct or substitution)
            if np.random.random() < ps:
                # Substitution error
                r.append(int(vi) ^ 1)  # Substitute vi with vi ⊕ 1
            else:
                # Correct transmission
                r.append(int(vi))

    return ''.join(map(str,r))

In [262]:
def get_alphas(received, N, M, n):

  alphas = np.zeros((n, M, 3)) #for every time step, save the M best alphas with (value, s' ,d')

  alphas[0][0] = [1,0,0] # set the first value to 0 (value,s',d')

  for t in range(1,n): # all iterations except first one
    s0 = np.mod(alphas[t-1][:,1],2*n+1) # finding all the s that we can lead to given the current state with b=0
    s1 = np.mod(alphas[t-1][:,1]+t,2*n+1) # finding all the s that we can lead to given the current state
    s = np.append(s0,s1) # both for b=0 and b=1 at that current bit.

    # isolating the unique values of s, and indexes of all previous states that lead to them
    # output is the unique s, and for every element in original s array what is the index of the unique value
    out, indices = np.unique(s, return_inverse=True)

    # creating all combinations of next states
    next_states = np.zeros((len(out),N+n,3)) # (output for each s, index of output for each t, (value,s,d)
    next_states[:,:,2] = np.arange(N+n)-n # create all combinations relating to s,d, including negative values

    # setting the s states using the unique s
    for i in range(len(out)):
      next_states[i,:,1] = out[i]


    # calculate all contributions of states

    for i in range(len(s)):
      #finding the bit b of contribution
      b=int(i>=len(s0))
      s_index = indices[i]
      s_tag_index = i%len(s0)
      for d in range(-n,N): # including the negative values
        next_states[s_index][d+n][0] += gamma(received,t,d,alphas[t-1][s_tag_index][2],b)*alphas[t-1][s_tag_index][0]
    # merge all states
    merged = np.concatenate(next_states,axis=0)
    # find M best states
    if (len(merged) < M):
      alphas[t][:len(merged)] = merged[merged[:,0].argsort()][-len(merged):]
      alphas[t][len(merged):] = np.zeros((M-len(merged),3))
    else:
      alphas[t] = merged[merged[:,0].argsort()][-M:]


    # renormalize
    sum = np.sum(alphas[t][:,0])
    if (sum != 0):
      alphas[t][:,0] = alphas[t][:,0]#/sum
  return alphas

In [263]:
a = 15
get_alphas("01001001",8,100,8)[-1]

array([[ 7.04023329e-03,  3.00000000e+00,  0.00000000e+00],
       [ 7.79236280e-03,  1.50000000e+01,  0.00000000e+00],
       [ 8.02085263e-03,  1.60000000e+01,  0.00000000e+00],
       [ 8.13826974e-03,  0.00000000e+00,  0.00000000e+00],
       [ 8.46015677e-03,  1.00000000e+00,  0.00000000e+00],
       [ 9.19151306e-03,  7.00000000e+00, -5.00000000e+00],
       [ 9.35173035e-03,  8.00000000e+00, -5.00000000e+00],
       [ 9.36603546e-03,  6.00000000e+00, -5.00000000e+00],
       [ 9.55200195e-03,  5.00000000e+00, -5.00000000e+00],
       [ 9.67693329e-03,  9.00000000e+00, -5.00000000e+00],
       [ 1.00383817e-02,  8.00000000e+00,  0.00000000e+00],
       [ 1.01952802e-02,  1.00000000e+01,  0.00000000e+00],
       [ 1.02796555e-02,  1.00000000e+01, -5.00000000e+00],
       [ 1.03442109e-02,  1.40000000e+01,  0.00000000e+00],
       [ 1.04621620e-02,  1.30000000e+01,  0.00000000e+00],
       [ 1.05095967e-02,  5.00000000e+00,  0.00000000e+00],
       [ 1.06258392e-02,  4.00000000e+00

In [264]:
def get_betas(received, N, M, n, alphas):
  betas = np.zeros((n, M, 3))

  new_betas = alphas.copy()
  new_betas[:,:,0] = 0 # replace relevant beta values

  # shifting betas by one value to represent 1->n and not 0->n-1
  # so betas[0] is actually the value at time t=1
  betas[:n-1] = new_betas[1:]

  # setting last beta
  betas[-1][0] = [1,a,N-n]

  mod_value = 2*n+1

  for t in range(n-2,-1,-1):
    prev_beta = [None,None,None]
    for b in betas[t]:
      if (b[1] == prev_beta[1] and b[2] == prev_beta[2]): # the exact same element in the current iteration, to deal with repeat elements.
        continue
      s_tag = b[1] # previous s
      d_tag = b[2] # previous d
      s0 = s_tag
      s1 = np.mod(s_tag+(t+2),mod_value)

      
      # finding all relevant states that might have a contribution
      relevant0 = betas[t+1][betas[t+1][:,1]==s0]
      relevant1 = betas[t+1][betas[t+1][:,1]==s1]
      
      
      for r in relevant0:
        #if (d_tag == -4 and t==n-2 and r[0] == 1):
         #   print(gamma(received,t+2,int(r[2]),d_tag,0))
        b[0] += gamma(received,t+2,int(r[2]),d_tag,0)*r[0]
      for r in relevant1:
        b[0] += gamma(received,t+2,int(r[2]),d_tag,1)*r[0]


      prev_beta = b

    # Normalize betas[t][:,0] if sum is non-zero
    sum = np.sum(betas[t][:,0])


    if (sum != 0 ):
      betas[t][:,0] = betas[t][:,0] #/  sum
    else: betas[t][:,0] = 0

  return betas

In [266]:
a = 15
get_betas("01001001",8,400,8, get_alphas("01001001", 8, 400, 8))[-4]

array([[ 4.21478035e-08,  0.00000000e+00, -8.00000000e+00],
       [ 0.00000000e+00,  1.50000000e+01,  6.00000000e+00],
       [ 1.64800440e-08,  8.00000000e+00, -8.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [246]:
get_betas("10101",5,200,5, get_alphas("10101", 5, 200, 5))[-1][:,0]

array([[ 1., 15.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.

In [249]:
a=15
np.sum(get_betas("10101",5,200,5, get_alphas("10101", 5, 200, 5))[-1][:,0])

1.0

In [237]:
def sequence_prob(received, t, b, n, alphas, betas):
  sum = 0
  for alpha in alphas[t-1]:
    s_tag = int(alpha[1])
    d_tag = int(alpha[2])
    for beta in betas[t-1][betas[t-1][:,1] == np.mod(s_tag + t*b,2*n+1)]:
      d = int(beta[2])
      g = gamma(received,t,d,d_tag,b)
      sum += g*alpha[0]*beta[0]
  return sum

In [238]:
def get_likelyhood(received_seq, n, M):
    # perform on one sequence at a time, helps to prevent multiple calculations
    avg = np.zeros(n)
    N = len(received_seq)
    # Step 1: Find alphas for sequence
    alphas = get_alphas(received_seq, N, M, n)
    # Step 2: Using alphas chosen s,d - find betas for sequence
    betas = get_betas(received_seq, N, M, n, alphas)
    
    # Step 3: Calculate sequence probability
    for t in range(1,n+1):
        prob0 = sequence_prob(received_seq,t=t,b=0,n=n,alphas=alphas,betas=betas)
        prob1 = sequence_prob(received_seq,t=t,b=1,n=n,alphas=alphas,betas=betas)
        if (prob0 == 0 or prob1 ==0): 
            # at least one of the probabilities is very low and omitted. Assume L=0 (equally likely) as stated in paper and rely on other sequences
            # This is done because of M, for full recovery we can assume full recovery
            L = 0
        else: L = np.log(prob0/prob1)
        avg[t-1] = L
    return avg

In [240]:
a = 15
get_likelyhood("01001001",8, 1000)

array([ 1.32915564,  0.54375653,  1.2331389 ,  0.71668835,  0.09505536,
        0.71825645,  0.55359556, -0.88260703])

In [191]:
def get_probs(received_seq, t, n, sequences, M):

  avg = np.zeros(n)
  # Iterate over all sequences
  for i in range(sequences):
    N = len(received_seq[i]) # length of received message
    # Step 1: Find alphas for sequence
    alphas = get_alphas(received_seq[i], N, M, n)

    # Step 2: Using alphas chosen s,d - find betas for sequence
    betas = get_betas(received_seq[i], N, M, n, alphas)


    #print(betas)
    # Step 2: Calculate sequence probability
    for t in range(1,n+1):
      prob0 = sequence_prob(received_seq[i],t=t,b=0,n=n, alphas=alphas, betas=betas)
      prob1 = sequence_prob(received_seq[i],t=t,b=1,n=n, alphas=alphas, betas=betas)
      if (prob1 == 0):
        prob = 100
      elif (prob0 == 0):
        prob = -100
      else: prob = np.log(prob0/prob1)
      avg[t-1] += prob
  #avg = avg/sequences
  #print(avg)
  return avg

In [143]:
 import random
 def rand_input(length):
    return ''.join(random.choice(['0', '1']) for _ in range(length))

In [136]:
rand_input(6)

'101100'

In [123]:
a = 4 # the number used for encoding
input = rand_input(60) # the given input sequence
pr = 3/100 # percent
p_d, p_i, p_s  = pr/3, pr/3, pr/3 #probabilities
p_t = 1-p_d-p_i #probability of transition
n = find_n(len(input)) # length of the coded number
sequences = 1 # number of generated sequences

In [124]:
def hard_decoder(received, a, n):
    N = len(received)
    digit_sum = np.sum(received)
    syndrome = sum((i+1) * bit for i, bit in enumerate(received))
    diff = np.abs(syndrome - a)
    if (np.abs(n-N) > 1 or diff > n+1):
        # beyond the scope of the algorithm
        return received
    
    if (N==n+1): # assume one element inserted
        if (diff > digit_sum): # 1 inserted
            index_0 = diff - digit_sum
            inserted_index = np.where(received == 0)[0][index_0-1]+1 # index where the extra element is
        elif (diff==0): # edge case where 0 was inserted at the end
            inserted_index = N-1
        elif (diff < digit_sum): # 0 inserted
            index_1 = -diff
            inserted_index = np.where(received == 1)[0][index_1]-1 # index where the extra element is
        else: # we don't know what was inserted but it is the first digit anyways
            inserted_index = 0
        return np.delete(received,inserted_index)
    if (N==n-1): # assume one element deleted
        deleted_element = 0
        if (diff > digit_sum): # 1 deleted
            deleted_element = 1
            index_0 = diff - (digit_sum + 1) # plus 1 because the 1 element was deleted
            if(index_0 == 0): # edge case where it is before any zeroes
                deleted_index = 0
            else: deleted_index = np.where(received == 0)[0][index_0-1]+1 # index where the element should be 
        elif (diff == 0): # 0 deleted, edge case where it was last element
            deleted_index = N
        elif (diff < digit_sum): # 0 deleted
            index_1 = -diff
            deleted_index = np.where(received == 1)[0][index_1]#-1 # all elements are shifted right after insert
        else: # has to be 0 that was deleted
            deleted_index = 0
        return np.insert(received,deleted_index,deleted_element)
    if (N==n): # Assume 1 substitution
        #if (diff == 0): # edge case of 0 at the end
            #recieved
        #print(diff)
        index = diff-1
        received[index] = 1- received[index]
        return received

In [125]:
print(hard_decoder(np.array([0,0,0,1,0,0,1,0,0,1]), a = 0, n = 10)) # [1, 0, 1, 1, 0, 0, 1]

[0 0 0 1 0 0 1 0 0 1]


In [127]:
def count_diff(str1,str2):
    len1, len2 = len(str1), len(str2)
    min_len = min(len1,len2)
    
    return sum(c1 != c2 for c1, c2 in zip(str1[:min_len], str2[:min_len])) + np.abs(len1-len2)

In [128]:
error_rates = [1.0, 1.5, 2.0, 2.5, 3.0]
BER = np.zeros((len(error_rates), 3))
tries = 200
n = 68
for e in error_rates:
  pr = e/100 # percent
  p_d, p_i, p_s  = pr/3, pr/3, pr/3 #probabilities
  p_t = 1-p_d-p_i #probability of transition
  BER_avg_200 = 0
  BER_avg_300 = 0
  BER_avg_hard = 0
  for t in range(tries):
    input = rand_input(60)
    coded = encoder_VT(input,a=0)
    received_seq = [print_strand(coded, p_i, p_d, p_s) for i in range(sequences)]
    
    extracted_200 = extract_message(''.join((get_probs(received_seq, t=1, n=n, sequences=sequences, M=200)<0).astype(int).astype(str)))
    extracted_300 = extract_message(''.join((get_probs(received_seq, t=1, n=n, sequences=sequences, M=300)<0).astype(int).astype(str)))
    extracted_hard = extract_message(''.join(np.array(hard_decoder(np.array(list(received_seq[0])).astype(int), a=0, n=68)).astype(str)))
        
    BER_avg_200 += count_diff(extracted_200, input)
    BER_avg_300 += count_diff(extracted_300, input)
    BER_avg_hard += count_diff(extracted_hard, input)
    print("t: ", t, " e:", e)
  BER[error_rates.index(e),0] = BER_avg_200
  BER[error_rates.index(e),1] = BER_avg_300
  BER[error_rates.index(e),2] = BER_avg_hard
  
BER /= (tries * 60) # normalizing

t:  0  e: 1.0
t:  1  e: 1.0
t:  2  e: 1.0
t:  3  e: 1.0


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.yscale("log")
plt.ylim(1e-2,1)
plt.grid()
plt.xlabel("Error Rate (%)")
plt.ylabel("BER")
plt.plot(error_rates, BER[:,0], label = "M=200")
plt.plot(error_rates, BER[:,1], label = "M=300")
plt.plot(error_rates, BER[:,2], label = "Hard Decision")
plt.legend()
plt.minorticks_on()
plt.show()


In [ ]:
error_rates = [1.0, 1.5, 2.0, 2.5, 3.0]
num_sequences = [1,3,5,10]
BER_sequences = np.zeros((len(num_sequences),len(error_rates)))
tries = 100



for e in error_rates:
    pr = e/100 # percent
    p_d, p_i, p_s  = pr/3, pr/3, pr/3 #probabilities
    p_t = 1-p_d-p_i #probability of transition
    
    # each time we will use 
    for t in range(tries):
        input = rand_input(60)
        # encoding
        coded = encoder_VT(input,a=0)
        received_probs = np.zeros((30,n))
        # recieving new strands and finding the probabilities
        for i in range(30): # for all attempts, 3*10 so that all sequences divide well
            received_probs[i] = get_likelyhood(print_strand(coded, p_i, p_d, p_s), n=68, M=200)
        # decoding based on sequence size
        for sequences in num_sequences:
            split_probs = np.split(received_probs,30//sequences)
            # calculate sum for each of the split likelyhoods and extract the message
            for i in range(len(split_probs)):
                sum_likelyhood = np.sum(split_probs[i],axis=0)
                # decoding and extracting
                extracted = extract_message(''.join((sum_likelyhood<0).astype(int).astype(str)))
                BER_sequences[num_sequences.index(sequences),error_rates.index(e)] += count_diff(extracted, input)
            # normalize
            BER_sequences[num_sequences.index(sequences),error_rates.index(e)]/= len(split_probs)
        print("e: ",e," t: ", t)
BER_sequences /= tries

In [ ]:
import matplotlib.pyplot as plot
plot.yscale("log")
plot.grid()
plt.ylim(1e-3,1)
plot.xlabel("Error Rate (%)")
plot.ylabel("BER")
plt.plot(error_rates, BER_sequences[0], label = "c=1")
plt.plot(error_rates, BER_sequences[1], label = "c=3")
plt.plot(error_rates, BER_sequences[2], label = "c=5")
plt.plot(error_rates, BER_sequences[3], label = "c=10")
plt.legend()
plt.minorticks_on()
plt.show()



---

